In [1]:
%env NOTIONPY_LOG_LEVEL=debug
import os, logging, uuid, urllib.parse, json, requests, glob, re
from pathlib import Path
from notion.client import NotionClient
from notion.block import TextBlock, PageBlock, HeaderBlock, Block
from notion.collection import CollectionRowBlock, CollectionViewBlock
# client = NotionClient(token_v2=os.environ['NOTION_TOKEN_V2']) # inspect browser cookies for token value
token_v2 = '694be7fb8e2a034287d59437b8a2dae791767d59f7240709ecf73e3b510c151a6b3af2d6b8f921bf7614b8ca75646558fef4795bb0d68ae9d43057560fa94f77119733a58d3e8d0648e2aee24273'
client = NotionClient(token_v2=token_v2)

env: NOTIONPY_LOG_LEVEL=debug


### Update lambda with new token

#### Experimenting

In [ ]:
import boto3
from dgnutils import time_dt_to_ck
_lambda = boto3.client('lambda')


In [ ]:
time_dt_to_ck(datetime.datetime.now())

In [ ]:
lambda_data = _lambda.get_function(FunctionName='notion-calendars', Qualifier='PROD'); lambda_data
version = lambda_data['Configuration']['Version']
environment = lambda_data['Configuration']['Environment']
code_sha_256 = lambda_data['Configuration']['CodeSha256'] # For verifying update

# lambda_data

In [ ]:
def test_kw(**kw):
    test = {'a':{'b':'c'}}; test
    test['a'].update(kw)
    print(','.join(kw.keys()))
    return test
test_kw(b=2, c=3)

In [ ]:
fn_name = 'notion-calendars'
new_version = '7'
response = _lambda.update_alias(
    FunctionName=fn_name,
    Name='PROD',
    FunctionVersion=new_version,
); response


In [ ]:
response

In [ ]:
response = _lambda.publish_version(
    FunctionName='notion-calendars',
    CodeSha256=code_sha_256,
    Description=f'Updated token on {time_dt_to_ck(datetime.datetime.now())}',
)
new_version = response['Version']

In [ ]:
token_v2 = '3b53dba83312d42555eb8056adae307a259020c2c527b4746f88054d6406376a2d0b8fb0ffba61dce37a07d3941314b6eb73c8543ed2b2bf949423584dd3ab51baa8fb887853fe8eff1dd24c1049'
environment = _lambda.get_function_configuration(FunctionName='notion-calendars')['Environment']; environment
environment['Variables']['NOTION_TOKEN_V2'] = token_v2
_lambda.update_function_configuration(
    FunctionName='notion-calendars',
    Environment=environment
)

#### Function

In [ ]:
def update_lambda_function_config(fn_name, alias='PROD', **new_args):
	_lambda = boto3.client('lambda')

	print(f'Getting data on current {fn_name}:{alias} lambda')
	lambda_data = _lambda.get_function(FunctionName=fn_name, Qualifier=alias)
	version = lambda_data['Configuration']['Version']
	environment = lambda_data['Configuration']['Environment']
	print(f'Current version is {version}')

	print(f'Updating $LATEST version with new token')
	environment['Variables'].update(new_args) # Put the keyword args (e.g. NOTION_V2_TOKEN='asdfasdf' into the new function)
	response = _lambda.update_function_configuration(
		FunctionName=fn_name,
		Environment=environment
	)
	code_sha_256 = response['CodeSha256']

	print(f'Publishing new version from $LATEST')
	response = _lambda.publish_version(
		FunctionName=fn_name,
		CodeSha256=code_sha_256,
		Description=f'Updated {", ".join(new_args.keys())} on {time_dt_to_ck(datetime.datetime.now())}',
	)
	new_version = response['Version']
	print(f'New version is {new_version}')

	print(f'Updating {alias} to point at $LATEST')
	response = _lambda.update_alias(
		FunctionName=fn_name,
		Name=alias,
		FunctionVersion=new_version,
	)
	newest_version = response['FunctionVersion']
	print(f'{fn_name}:{alias} is now pointing at {newest_version}')

In [ ]:
# if __name__ == "__main__":
fn_name = 'notion-calendars';
# parser = argparse.ArgumentParser(description='Notion calendars token updater. Pass a token via -t to update the token in lambda')
# parser.add_argument('-t', '--token', action="store", help='Notion token_v2. Can be found by inspecting cookies from a logged-in session of notion.so')
# args = parser.parse_args()
update_lambda_function_config(fn_name, alias='PROD', NOTION_TOKEN_V2='asdf')

### Scrape for Notion Token

In [ ]:
import requests

In [ ]:
session = requests.Session()

In [ ]:
login_url = 'https://www.notion.so/login'
result = session.get(login_url)
login_tree = bs4.BeautifulSoup(result.text, 'html.parser'); login_tree
if not result.ok: raise ConnectionError(result.status_code)
    

In [ ]:
payload = {
        "user[email]": os.environ['CK_USER'], 
        "user[password]": os.environ['CK_PASSWORD'],
        "authenticity_token": login_tree.find("input", {"name":"authenticity_token"})['value']
    };

In [ ]:
def get_ck_session():
    """Handle the initial page load and session creation
    Returns an authenticated session"""
    
    session = requests.session()
    login_url = "https://app.convertkit.com/users/login"

    result = session.get(login_url)
    login_tree = bs4.BeautifulSoup(result.text, 'html.parser'); login_tree
    if not result.ok: raise ConnectionError(result.status_code)

    payload = {
        "user[email]": os.environ['CK_USER'], 
        "user[password]": os.environ['CK_PASSWORD'],
        "authenticity_token": login_tree.find("input", {"name":"authenticity_token"})['value']
    };

    login_result = session.post(login_url, data = payload, headers = dict(referer=login_url)); 
    if not login_result.ok: raise ConnectionError(login_result.status_code)
    return session


### Updating Boards

In [ ]:
client.get_top_level_pages()
board = client.get_collection("d23acfdd-e88f-4f8c-9e2e-c24d18435bf8")
board_view = client.get_collection_view("https://www.notion.so/declannishiyama/410d2ce25a534ab6a9d3b8b85f485b2d?v=a7c7b022f3314eb3a168574ab9427780")

In [ ]:
??board_view.build_query

In [ ]:
board_view.default_query().execute()

In [ ]:
board_view.get("query2")

In [ ]:
filter_params = {'filter': {'filters': [{'filter': {'value': {'type': 'exact', 'value': '1'},
     'operator': 'string_contains'},
    'property': 'title'}],
  'operator': 'and'},
 'group_by': 'X_.t',
 'aggregations': [{'aggregator': 'count'}]}
board_view.build_query(filter=filter_params).execute()
# board_view.build_query(group_by='wV?$').execute()

In [ ]:
??board_view.build_query

### Updating Projects

In [ ]:
temp_page = client.get_block("https://www.notion.so/declannishiyama/Temp-fc11adcd7c814418838f6925a11becc9")

In [ ]:
# Get project list
client.get_top_level_pages()
gtd = client.get_collection("30466763-e37a-4d6d-91a2-d7209556e366"); gtd
properties = [[i, g['name'], g['id']] for i, g in enumerate(gtd.get_schema_properties())]; properties
project_names = [o['value'] for o in gtd.get_schema_property("Project")['options']]; project_names

project_to_note_dict = {}
project_has_plan_dict = {}
for g in gtd.get_rows():
    # All note data
    project_to_note_dict.setdefault(g.get_property("Project"), []).append(g.title)
    
    # Project plan info
    g_plan_bool = g.title.lower().find("plan") >= 0 and g.get_property("Type") == "Project Support"
    g_project = g.get_property("Project")
    project_has_plan_boolean = project_has_plan_dict.setdefault(g_project, False) or g_plan_bool
    if project_has_plan_boolean:
        project_has_plan_dict[g_project] = True
        
# Check for missing projects
all_projects = [g['value'] for g in gtd.get_schema_property("Project")['options']]
for project in all_projects:
    if project not in project_has_plan_dict: project_has_plan_dict[project] = False

In [ ]:
# get list of projects without a plan
projects_without_plans = [k for k,v in project_has_plan_dict.items() if k and not v]; projects_without_plans

# page = project_management_page
# cvb = page.children.add_new(CollectionViewBlock)
# cvb.collection = client.get_collection(
#     client.create_record("collection", parent=cvb, schema=get_collection_schema(project_list))
# )
# cvb.title="My data!"
# view = cvb.views.add_new(view_type="table")

# All projects and notes within
for i, project_name in enumerate(projects_without_plans): 
    print(f'\r Updating {i}/{len(projects_without_plans)-1}', end='')
    if project_name == "None": continue
    row = gtd.add_row()
    row.name = project_name + " Plan"
    row.type = "Project Support"
    row.project = project_name
    row.context = "None"
    row.children.add_new("table_of_contents")
    row.children.add_new(HeaderBlock, title="Outcome")
    row.children.add_new(HeaderBlock, title="Brainstorm")

In [ ]:
# List out projects and notes within
for project_name, has_plan_bool in project_has_plan_dict.items():
    if project_name == "None": continue
    if not has_plan_bool:
        project_block = project_management_page.children.add_new(PageBlock, title=project_name)
#     for note in project_to_note_dict.get(project_name, []):
#         project_block.children.add_new(TextBlock, title=note)

In [ ]:
def get_collection_schema(project_list):
    return {
        "31!t": {
            "name": "Context", 
            "type": "multi_select",
            "options": [{"color":"default", "id": str(uuid.uuid4()), "value": "None"}]
        },
        "U'\\G": { 
            "name": "Project", 
            "type": "select", 
            "options": [{"color":"default", "id": str(uuid.uuid4()), "value": project} for project in project_list]
        },
        "}V#g": {
            "name": "Type",
            "type": "select",
            "options": [{"color":"default", "id": str(uuid.uuid4()), "value": "Project Support"}]
        },
        "title": {"name": "Name", "type": "title"},
    }

### Python API Experimenting

In [ ]:
# Get all projects
gtd_url = "https://www.notion.so/declannishiyama/dec049b9497f4b269a33220b541f079b?v=1a2336854d754a04b75ad036a232091c"
gtd = client..get_collection_view(calendar_url)
calendar_query = calendar.build_query()
calendar_entries = calendar_query.execute()
len(calendar_entries)

collection = calendar.collection
schema = collection.get_schema_properties()

In [ ]:
test_page = client.get_block("https://www.notion.so/declannishiyama/The-title-has-now-changed-and-has-live-updated-in-the-browser-55fff8671b9344a6a73a1bf038a9332f")

In [ ]:
test_page.children[0].title

In [ ]:
# Replace this URL with the URL of the page you want to edit
page = https://www.notion.so/myorg/Test-c0d20a71c0944985ae96e661ccc99821")

print("The old title is:", page.title)

# Note: You can use Markdown! We convert on-the-fly to Notion's internal formatted text data structure.
page.title = "The title has now changed, and has *live-updated* in the browser!"

In [ ]:
def my_callback(record, difference):
    print(f"The record's title is now: {record.title}")
    print("Here's what was changed:")
    print(difference)

# move my block to after the video
test_page.add_callback(my_callback)

### Collecting CSV Data into Dataframe

#### Book Data

In [ ]:
from datetime import datetime
import pandas as pd

In [ ]:
text="The Brothers Karamazov (1-5-20 to 1-20-20)\nChristianity for People who aren't Christians (1-1-20)\nGet Together (12-24-19)\nPersonal Kanban (12-11-19 to 12-18-19)\nScientific Advertising (11-2-19 to 12-8-19)\nAtomic Habits (10-10-19 to 12-4-19)\nSmart people should build things (11-25-19 to 12-3-19)\nMr. Penumbra's 24-Hour Bookstore (11-22-19)\nStart Your Own Corporation (10-31-19)\nThe Rise and Fall of the Dinosaurs (10-25-19)\nAltered Traits (10-22-19)\nPirate Latitudes (10-13-19)\nRich Dad, Poor Dad (10-10-19)\nHow to Traumatize Your Kids (9-19-19)\nThe Mind Illuminated (9-10-19)\nSurvivor (8-20-19)\nCribsheet (8-17-19)\nEpic: The Story God is Telling… (8-15-19)\nPlay it Loud (8-11-19)\nExpecting Better (8-7-19)\nFarewell to Manzanar (7-31-19)\nHow to Read Wittgenstein (7-29-19)\nKid Gloves (7-22-19)\nFrom Gutenberg to Google (7-21-19)\nFinding Zero (7-10-19)\nI am a Strange Loop (7-6-19)\nThe Brief Wondrous Life of Oscar Wao (6-14-19)\nWhere’d you go Bernadette? (5-30-19)\nHow to be Happy (5-28-19)\nAge of Ambition (5-23-19)\nThe Gifts of Imperfection (4-24-19)\nHow to Have Your Second Child First (4-20-19)\nWaking Up (4-10-19)\nDeath’s End (3-21-19)\nZen and the Art of Motorcycle Maintenance (3-15-19)\nThe Cursed Child (2-24-19)\nThe Expectant Father (2-21-19)\nGödel, Escher, Bach (1-6-19)\nKafka (10-21-18)\nFoundation and the Empire (10-17-18)\nThe Foundation (9-17-18)\nSilent Spring (9-13-18)\nFlowers for Algernon (8-6-18)\nGod: A Human History (7-5-18)\nThe Dark Forest (7-2-18)\nHit Refresh (7-1-18)\nThe Three Body Problem (6-14-18)\nHow to Change Your Mind: Psychedelics (6-10-18)\nThe Dark Tower (5-29-18)\nThe Four Agreements (5-27-18)\nOn Writing (2-24-18)\nThe Design of Everyday Things (2-12-18)\nThe Art of Living (2-9-18)\nMostly Harmless (1-9-18)\nYoung Zaphod Plays it Safe (1-7-18)\nSo Long and Thanks for all the Fish (1-7-18)\nWhy Buddhism is True (1-6-18)\nLife, the Universe and Everything (12-27-17)\nThe Restaurant at the End of the Universe (12-20-17)\nOf Mice and Men (12-17-17)\nPrinciples, Work (12-15-17)\nHitchhiker’s Guide to the Galaxy, Book 1 (12-9-17)\nPrinciples, Life (12-6-17)\nRomeo and Juliet (11-12-17)\nThe Four (10-27-17)\nSeven Brief Lessons on Physics (10-17-17)\nAnimal Farm (10-12-17)\nSuperintelligence (10-10-17)\nThe Once and Future Liberal (9-15-17)\nHillbilly Elegy (9-3-17)\nZero to One (9-2-17)\nHandmaid’s Tale (8-11-17)\nSapiens (6-19-17)\nA Rose For Emily (5-13-17)\nThe Stranger (4-29-17)\nChecklist Manifesto (4-22-17)\nThe Art of Product Management (4-16-17)\nThe Wind-Up Bird Chronicle (4-7-17)\nThinking Fast and Slow (3-22-17)\nFahrenheit 451 (2-11-17)\nAn Astronaut's Guide to Life on Earth (2-9-17)\n1984 (2-7-17)\nDecode and Conquer (1-31-17)\nThe Everything Store (1-12-17)\nThe Amazon Way (1-2-17)\nWatership Down (12-31-16)\nSeven Habits of Highly Effective People (6-7-16 to 9-5-16)\nCase in Point (5-5-16 to 6-6-16)\nThe Third Wave (6-1-16 to 6-5-16)\nSmarter, Faster, Better (4-24-16 to 5-5-16)\nHow to Create a Mind (4-10-16 to 4-23-16)\nThe Elements of Style (3-21-16 to 3-27-16)\n21 Irrefutable Laws of Leadership (2-28-16 to 3-20-16)\nMachines of Loving Grace (10-4-16 to 2-28-16)\nScott Pilgrim’s Precious Little Life (2-21-16 to 2-21-16)\nPredictably Irrational (1-17-16 to 2-20-16)\nThe Power of Habit (12-24-15 to 12-29-15)\nFailure to Learn (11-4-15 to 11-15-15)\nBenjamin Franklin’s Biography (10-4-15)\nStructures (9-29-15)\nHitchhiker’s Guide to the Galaxy (9-2-15)\nSuperFreakonomics (7-27-15 to 8-5-15)\nFreakonomics (7-5-15 to 7-24-15)\nHow to Win Friends and Influence People (8-26-14)\nBeyond the Zonules of Zinn (2-24-13)\nThe Mind’s New Science (3-17-13)\nConsciousness: Confessions of a Romantic Reductionist (5-4-13)\nHow to Read a Book (9-15-13)\nConnectome (5-12-13 to 6-1-13) \nOn Intelligence (1-10-13)\nIn Search of Memory (12-15-12 to 1-5-13)\nIncognito (2-27-13)"

In [ ]:
book_data = []
for t in text.split('\n'):
    start = ""; end = ""
    loc1 = t.find(" (")
    loc2 = t.find(" to ", loc1)
    loc3 = t.find(")")
    title = t[:loc1]
    
    if loc2 > 0: # if there is 'to'
        start = t[loc1+2:loc2]
        end = t[loc2+4:loc3]
    else:
        end = t[loc1+2:loc3]
    book_data.append([title, start, end])
print(book_data)

In [ ]:
df = pd.DataFrame(book_data, columns = ['Title','Start', 'Due']) 

In [ ]:
%pwd

In [ ]:
df.to_csv('test')

#### Ice Cream Journal

In [ ]:
# html2markdown use this next time

In [ ]:
import mistletoe
from md2notion.upload import upload, uploadBlock
from md2notion.NotionPyRenderer import NotionPyRenderer
from notion.block import PageBlock
import bs4
ice_cream_page = """<body id="tinymce" class="mce-content-body " data-id="elm1" contenteditable="true"><h3><b>Avocado</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">David Lebovitz’ Recipe&nbsp;</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Good flavor</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Medium icy texture</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Tasted like banana ice cream</span></li></ul><h3><b>Bacon</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Flavor was mediocre</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Bacon bit taste was too strong</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The base flavor was mediocre</span></li></ul><h3><b>Banana</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><a href="https://altonbrown.com/banana-ice-cream-recipe/" data-mce-href="https://altonbrown.com/banana-ice-cream-recipe/"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Alton Brown’s recipe</span></a></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Good flavor</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Medium icy texture</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Remember to peel bananas and then freeze them. Let them ripen before freezing (many brown spots)</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Or let them thaw before peeling</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">6/2/15 Made another Batch</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">This time I let the banana’s thaw for a very long time. Maybe 3 hours. The recipe suggests ~1 hour.</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">At the end of the thaw there was brown liquid in the bag—It might have been very flavor ladened but I also think it would have made the ice cream icy so I left it out.</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">I doubled the amount of corn syrup (a ton), vanilla bean, heavy cream to lessen the banana flavor and make it more creamy (wanted to reduce iciness)</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The churn went for 30 minutes and was well whipped afterwards. I use a lot of ice and ice cream salt.</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">I also added spiced rum to taste. I ended up adding about 2 tablespoons.</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Will update on the ice cream later, but so far the flavor is somewhat mild and it is extremely creamy.&nbsp;</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">I also made a sauce to go along with it—the leftover banana juice, brown sugar, butter, and rum. It had a congealed layer on top from the butter so I may want to rethink that aspect.</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">10/5/17 Made another batch</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">I couldn’t tell whether to use 2.25 pounds of bananas or bananas with skin so I did the latter. I left the banana juice in the ziplock. I did it in the nutribullet, but it was just a little too much, I had to do two batches. I used less than 1 tbsp of spiced rum. I used an old dried up vanilla bean. I used a little extra heavy cream and only ½ cup of corn syrup (versus ¾ cup). I used 1.5 Tbsp of lemon juice also.</span></li></ul></ul><h3><b>Blackberry</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Pioneer woman recipe</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Made twice</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">First time wasn’t great</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used too much blackberry juice</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Second time was amazing</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Tasted mixture to ensure good juice ratio</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Medium icy texture</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Great Flavor</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Third time making this on 7-18-15. Doubled the recipe. Only had 8 egg yolks vs recommended 12</span></li></ul><h3><b>Blueberry</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">I used the Pioneer woman recipe, but used hand picked blueberries</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">I also reduced the blueberry liquid down a lot, which helped with the iciness</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Very smooth texture with little crunchies from small bits of frozen blueberry</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Great Flavor</span></li></ul><h3><b>Blueberry Cheesecake</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><a href="http://www.takeamegabite.com/blueberry-cheesecake-ice-cream/" data-mce-href="http://www.takeamegabite.com/blueberry-cheesecake-ice-cream/"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Internet Recipe</span></a></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Christine made and ate this one</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Probably wouldn’t make it again</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used ½ cup goats cheese (less than required)</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Tiny bit of a strange flavor (goat cheese flavor was very strong)</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Crumbs were best part</span></li></ul><h3><b>Blueberry-Lemon-Thyme</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">David Lebovitz vanilla recipe for base</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Except for vanilla bean</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Texture was good besides small chunks of thyme and lemon and blueberries</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">May want to food process them beforehand next time</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Flavor was delicious (only used half of the thyme)</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Color was a little gray, could add blue next time</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><a href="http://hipfoodiemom.com/2013/06/19/blueberry-lemon-and-thyme-ice-cream/" data-mce-href="http://hipfoodiemom.com/2013/06/19/blueberry-lemon-and-thyme-ice-cream/"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">http://hipfoodiemom.com/2013/06/19/blueberry-lemon-and-thyme-ice-cream/</span></a></li></ul><h3><b>Bourbon and Salted Buttered Pecans</b></h3><p><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Recipe: From Jeni’s book in the section for sundaes.</span></p><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The texture was good. A little icy</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Pecans were good</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The alcohol may have been a little too strong</span></li></ul><h3><b>Captain Crunch</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">First batch for Katie and David</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Made this using 1 cup milk and 1 cup heavy cream in a pan with 2 cups of cap’n crunch</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Heated that a little and then let sit for 30 minutes</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Strained all the cereal out</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Then added ~½ cup of sugar to taste and a pinch of salt</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Then added 5 egg yolks and made into a custard and poured that into 1 cup cold heavy cream</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Katie Woods ate the ice cream and cried</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Second batch for DJ and Loralee</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">1 cup milk, 1 cup heavy cream, 2 cups captain crunch. Heated until steam, then left covered for ~1hr</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Added ~¼ cup milk to thin out</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Added ~¼ cup sugar to taste</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">5 eggs + ~⅛ cup sugar</span></li></ul></ul><h3><b>Caramelized Banana-Brown Sugar</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Somewhat icy; not as creamy as others. Could use heavy cream vs. coconut milk</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Good flavor</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Doubled the recipe; made way too much</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><a href="http://www.davidlebovitz.com/2010/03/banana-ice-cream-recipe/" data-mce-href="http://www.davidlebovitz.com/2010/03/banana-ice-cream-recipe/"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">http://www.davidlebovitz.com/2010/03/banana-ice-cream-recipe/</span></a></li></ul><h3><b>Chai Tea</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><a href="http://www.simplyrecipes.com/recipes/chai_ice_cream/" data-mce-href="http://www.simplyrecipes.com/recipes/chai_ice_cream/"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">http://www.simplyrecipes.com/recipes/chai_ice_cream/</span></a></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Let the milk steep for 4 hours instead of 1 hour.&nbsp;</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The flavor may have been too strong</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Could have added more heavy cream</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The texture was perfect</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Superbowl 2017 Atlanta and Patriots</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Doubled the recipe</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used 2 cups milk initially instead of milk and heavy cream</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Accidentally put the black tea in too early. Didn’t know whether to use liquid tea or tea “leaves”</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Ended up making a full container, which is probably 2.5 quarts</span></li></ul></ul><h3><b>Chocolate</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Alton Brown’s Recipe</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Taste was great</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Texture was great</span></li></ul><h3><b>Cinnamon</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><a href="http://www.davidlebovitz.com/2013/11/cinnamon-ice-cream-recipe/" data-mce-href="http://www.davidlebovitz.com/2013/11/cinnamon-ice-cream-recipe/"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">http://www.davidlebovitz.com/2013/11/cinnamon-ice-cream-recipe/</span></a></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The recipe is also in the pdf version of the book</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used Canela with normal cinnamon sticks</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Followed the recipe instructions, but also doubled the ingredients. Once doubled it was almost too big to fit in the new containers (2 quarts)</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The custard tasted very subtle of cinnamon, more of just vanilla, but the final product is very cinnamony</span></li></ul><h3><b>Cinnamon Oreo</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Made cinnamon ice cream from http://www.davidlebovitz.com/2013/11/cinnamon-ice-cream-recipe/</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used canela cinnamon (mexican cinnamon)</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Also added about 2 teaspoons of ground cinnamon</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Added two crushed (nearly atomized) oreos to the chilled mixture</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Then added crumbled (rough crumble) 1 cup oreos to the churner</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The oreo cookies became soft in the final ice cream. May need to add later</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The oreo cream in the ice cream was the best part. May consider only adding that.</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The cinnamon flavor may have been a little too strong</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Made again on 2-28-16. This time I bought the bulk Canela from the mexican grocery store. I used the mallet to pulverize it.</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Crushed oreo cookies into the ice cream and kept the cream separate. Added the sprinkled chocolate into the mixture while it was becoming a custard, but then it was strained out. Added more to get a look after straining.</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The machine froze up really early this time! I set the timer for 40 minutes, but well before that it had stopped.</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The canella flavor was very strong, almost bordering on the too strong side. I used two long canella sticks from the self-serve at the Mexican grocery store. I’ll need to remember to use a little less next time.</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">I crushed up 6 oreos for the mixture throughout and also on top of the finished ice cream. Putting the oreos in at the last minute before packing helped to ensure they weren’t soggy.</span></li></ul></ul><h3><b>Cinnamon Toast Crunch</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">1st batch</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Made this using 1 cup milk and 1 cup heavy cream in a pan with ~4 cups Cinnamon Toast Crunch</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Heated to a steam and then let sit for 30 minutes</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Strained all the cereal out</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Added milk to thin out</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Added ~½ cup sugar to taste</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Added a pinch of salt</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Then added 5 egg yolks and made into a custard and poured that into 1 cup cold heavy cream</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">2nd batch: for Dinner with Katie and Alex on 4-19-17</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used 2 cups milk and ~4 cups CTC. I used the colander to get out all the cinnamon sugar from the box.</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Heated and let sit for just 15 minutes</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Strained all the cereal out</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Added ~½ cup sugar to taste and a pinch of sale</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Added 9 egg yolks and made into a custard then poured into 1 cup cold heavy cream</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">I think the 9 eggs was too many and that it started to scramble them.</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">3rd batch: On 3-5-18 for Raleigh friend visit.</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Followed first batch instructions, but added the cereal sugar at the end. It didn’t make a lot of volume. I think I should use the 2 cups of milk next time instead of milk and heavy cream.</span></li></ul></ul><h3><b>Coconut</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">First recipe (Vegan and coconut-milk based): </span><a href="http://www.thekitchn.com/how-to-make-vegan-ice-cream-cooking-lessons-from-the-kitchn-204755" data-mce-href="http://www.thekitchn.com/how-to-make-vegan-ice-cream-cooking-lessons-from-the-kitchn-204755"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">http://www.thekitchn.com/how-to-make-vegan-ice-cream-cooking-lessons-from-the-kitchn-204755</span></a></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Ingredients</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">2 (13- to 15-ounce) cans full-fat coconut milk</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">1/2 cup sweetener, such as agave syrup, maple syrup, honey, turbinado sugar, or cane sugar</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">1/4 teaspoon salt</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">2 tablespoons cornstarch, or 1 tablespoon arrowroot starch</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">1 1/2 teaspoons vanilla extract</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Optional extras: nuts, chocolate (or carob) chips, fruit puree, cacao nibs, etc.</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">My notes</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used ½ cup corn syrup and 2 tablespoons of cornstarch</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Added ¾ cup sugar for sweetness</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The texture before churning looked really creamy</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Result was a little strange flavor, most likely from the coconut milk, but the flavor was good overall. Definitely more icy than the other ice cream</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Second make: 9-5-18</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Replaced half cup of coconut milk with coconut cream. So much creamier!</span></li></ul></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Second Recipe (Traditional ice cream)</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><a href="https://www.williams-sonoma.com/recipe/coconut-ice-cream.html" data-mce-href="https://www.williams-sonoma.com/recipe/coconut-ice-cream.html"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">https://www.williams-sonoma.com/recipe/coconut-ice-cream.html</span></a></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Ingredients:</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">1/4 cup firmly packed sweetened flaked coconut</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">1 1/2 cups half-and-half</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">3/4 cup canned sweetened cream of coconut</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">6 egg yolks&nbsp;</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">1/3 cup sugar</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">1 1/2 cups heavy cream</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Result was extremely creamy! The coconut flavor wasn’t as intense as I wanted it to be.</span></li></ul></ul><h3><b>Coffee</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Internet Recipe</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Flavor was great</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Texture was good</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Hard to find a good time to eat this one</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">1-16-18 Made the Van Leuvym recipe for Josie’s birthday</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The freeze dried coffee was a little expensive and I only used 1/100th of the container. I’ll need to keep a drawer for ice cream materials.</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The ice cream came out pretty amazing. Very creamy and tasty. It has a good level of bitterness.</span></li></ul></ul><h3><b>Coffee and Beer Ice Cream</b></h3><p><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Recipe:</span><a href="http://beingthesecretingredient.blogspot.com/2012/06/at-home-with-jenispart-two-kona-stout.html" data-mce-href="http://beingthesecretingredient.blogspot.com/2012/06/at-home-with-jenispart-two-kona-stout.html"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">http://beingthesecretingredient.blogspot.com/2012/06/at-home-with-jenispart-two-kona-stout.html</span></a><span style="font-weight: 400;" data-mce-style="font-weight: 400;"> or from Jeni’s book that Rachel and Aaron got us</span></p><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Doubled the recipe, but only used 2.5 ounces of cream cheese</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Ground up the coffee beans that Christine had—I think they were Starbucks brand.</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">I sifted out the smaller pieces and then let the mixture steep in the milk for quite a while.</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used Young’s double chocolate stout—the beer flavor was very subtle</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The texture turned out OK. Still a hint of icy-ness</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Everyone really liked it. Aaron especially enjoyed the flavor.</span></li></ul><p><br></p><h3><b>Cones</b></h3><p><span style="font-weight: 400;" data-mce-style="font-weight: 400;">David Lebovitz pg 228</span></p><h3><b>Cookie Butter</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">1st batch around 10/1/18, Christine said it was the best ice cream I’d ever made! I had forgotten to put the vanilla in at the right time so put it in late but besides that it went well. I used speculoos and cookie butter from trader Joes.&nbsp;</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">2nd batch I made on 11/21/18. I used biscoff cookies and biscoff cookie butter. I left the cookie butter and mixture on the eye for a long time and it started to thicken a lot. The texture was very thick before I even put in the eggs. The thermometer was reading 150 and the eggs looked like they were cooking so I took it out early. We’ll see how it turns out.</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Brown Eyed Baker recipe</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">For the Ice Cream:</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">1 cup (240 ml) whole milk</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">¾ cup (149 grams) granulated sugar</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">½ cup (120 grams) cookie butter</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">2 cups (480 ml) heavy cream, divided</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Pinch of salt</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">6 egg yolks</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">2 teaspoons vanilla extract</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">½ cup coarsely chopped speculoos cookies, about 12 cookies</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">For the Cookie Butter Swirl:</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">½ cup (120 grams) cookie butter, melted and cooled slightly</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><br></li></ul><h3><b>Cookie Dough</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Recipe: </span><a href="http://www.foodiemisadventures.com/2012/08/cookie-dough-ice-cream.html" data-mce-href="http://www.foodiemisadventures.com/2012/08/cookie-dough-ice-cream.html"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">http://www.foodiemisadventures.com/2012/08/cookie-dough-ice-cream.html</span></a></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used Nestle cookie dough</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">added two tablespoons of bourbon. The ice cream was creamy, but it too much bourbon flavor</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used the david lebovitz vanilla recipe instead of the cookie dough recipe from this link</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Made about one quart</span></li></ul><h3><b>Coriander Ice Cream</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Made this according to David Lebovitz recipe but used ⅔ the cream cheese to reduce cheese flavor</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Also couldn’t find any coriander essential oil so I toasted the seeds and then smashed them to infuse the flavor in the milk.</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">I think it turned out pretty successful except there was not very much made. Maybe 2 bowls worth. Should definitely increase the recipe size next time</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The raspberry sauce recipe said to go to 220 F but I couldn’t make it there and it ended up being a raspberry jam. I think Sean/Christine/Kelly didn’t mind it or even liked it that way</span></li></ul><h3><b>Creme Fraiche</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">I made this in the mountains on 6/7/18</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">This is a recipe from David Leibovitz as part of the peaches and cream combo. I did the buttermilk+heavy cream to make creme fraiche and would like to use the real thing next time</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The ice cream was a good texture but there were some frozen chunks in it. I had put it in the freezer before churning to cool it down. It might have had ice crystals that led to the chunks, or it is from the sides of the machine</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Christine said this was one of her favorite ice creams</span></li></ul><h3><b>Dark Chocolate</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Made it when Christine was 8 months pregnant, not sure the recipe.&nbsp; Maybe Lebovitz?</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Trying again with </span><a href="https://www.epicurious.com/recipes/member/views/the-best-dark-chocolate-ice-cream-ever-50151760" data-mce-href="https://www.epicurious.com/recipes/member/views/the-best-dark-chocolate-ice-cream-ever-50151760"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">https://www.epicurious.com/recipes/member/views/the-best-dark-chocolate-ice-cream-ever-50151760</span></a><span style="font-weight: 400;" data-mce-style="font-weight: 400;"> on 11/7/19</span></li></ul><h3><b>Eggnog</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Didn’t use a good recipe it was too icy.</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Could consider using straight eggnog or adding nutmeg to vanilla ice cream</span></li></ul><h3><b>Frozen Yogurt: Lemon w/ Berry sauce</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used the recipe from Jenni’s ice cream book</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Flavor and look were both amazing.&nbsp;</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The texture was perfect as well.</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Did not taste very yogurty. Very tart flavor from the lemon though.</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The cream cheese flavor was very strong and evident in the ice cream.</span></li></ul><h3><b>Frozen Yogurt: Lime w/ Rhubarb Compote</b></h3><p><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Recipe: From Jeni’s book in the section for sundaes.</span></p><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The texture was good. The yogurt make was about double so I could have used divided and made two services</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The flavor was bitter. I think this was because I went to far down on the zest (too much white on the zest) and from leaving it in for so long</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Rhubarb compote had a good taste but it was pretty bitter.</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The texture of the rhubarb was weird also. Also stringy</span></li></ul></ul><h3><b>Frozen Yogurt: Plain</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used modified David Leibovitz’s recipe</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">3 cups Greek Yogurt</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">¾ cup sugar</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">No vanilla</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">¼ teaspoon xanthum gum</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Had to combine the xanthum gum with the sugar and slowly add to the greek yogurt to prevent it from gumming up too much</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">I tried to make it without rock salt (used some near the end of churning) and only churned for 25 minutes</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Sweetness was good. Texture was super creamy (almost beyond necessary) right after churning.&nbsp;</span></li></ul><h3><b>Frozen Yogurt: Strawberry</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">I made this when Katie and Sam came in for Merlefest ~4/25/19</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used modified David Leibovitz’s recipe</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">1 pound (450g) strawberries, rinsed and hulled</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">2/3 cup (130g) sugar</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">optional: 2 teaspoons vodka or kirsch (i used white wine)</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">2 cup Greek-style yogurt (recipe calls for 1)</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">1 teaspoon fresh lemon juice</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The texture was great right after churning but was terribly icy afterwards. I need to try this with Xanthum Gum. And I want to try to reduce down the fruit instead of putting it in with all the watery content.</span></li></ul><h3><b>Frozen Yogurt: Vanilla</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used David Leibovitz’s recipe</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">It was too sweet. Could have used half as much sugar. Texture was great right after churning, but then got icy the next day. Might need to increase fat content or add heavy cream or something</span></li></ul><h3><b>Green Tea</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">David Lebovitz’ Recipe</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">First time followed the recipe exactly and was delicious</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Second time used too much green tea powder and the ice cream was too bitter</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Third time on 1/19/2020 replaced ⅓ sugar with corn syrup- equal portion (lebovitz said to use ¾ corn syrup for each unit of sugar. Online said more corn syrup</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Hard ice cream. Should look for some way to soften it</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Made twice</span></li></ul><p><br></p><h3><b>Guinness-Milk Chocolate Ice Cream</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">David Lebovitz’ Recipe</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used Hershey’s chocolate</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Maybe a different brand next time.</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Pieces too big and did not melt all the way</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Did not make a very large amount of ice cream</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Need to determine an alcohol to add next time. Should get some vodka.</span></li></ul><p><br></p><h3><b>Horchata</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><a href="http://www.seriouseats.com/recipes/2013/06/horchata-mexican-almond-drink-ice-cream-recipe.html" data-mce-href="http://www.seriouseats.com/recipes/2013/06/horchata-mexican-almond-drink-ice-cream-recipe.html"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">http://www.seriouseats.com/recipes/2013/06/horchata-mexican-almond-drink-ice-cream-recipe.html</span></a></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">I let the custard go for a very long time because it was very thin. It ended up being thick and the ice cream was creamy, but I boiled the mixture and think that I may have scrambled some of the eggs.</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">First taste is a little weak, but the aftertaste is very strong with cinnamon, vanilla and almond</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Very creamy. Could use rum next time for an alcohol</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Next time I could add a cinnamon swirl in it. Would decrease classiness but might make taste stronger</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Appearance could be improved with some cinnamon specks or flakes. Dont want to disrupt the texture too much.</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">2nd Attempt:</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used full amount of almonds (sliced), washed rice first (then dried), did not break canella (cinnamon stick), added a tablespoon cinnamon.</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">added ½ teaspoon vanilla and vanilla bean pods</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">added ½ teaspoon spiced rum</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">3rd Attempt: Fixed it again using the serious eats recipe again. Doubled the recipe. It was icy. The rice flavor wasn't strong. The almond flavor wasn't strong either. Might want to slice the almonds next time. Maybe blending some of the rice I. Would make the flavor more pronounced</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><a href="https://www.allrecipes.com/recipe/141828/lolas-horchata/" data-mce-href="https://www.allrecipes.com/recipe/141828/lolas-horchata/"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">https://www.allrecipes.com/recipe/141828/lolas-horchata/</span></a><span style="font-weight: 400;" data-mce-style="font-weight: 400;"> is a good strategy to improve rice flavor</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">4th Attempt:&nbsp;</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">I want to get the flavor stronger and the fat content higher, and use xanthum gum</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">1.5 cups 2% milk with 1 cup white long-grain rice blended for 30s on low then 30s on high. Let sit 15min</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Crushed ½ cup almonds and toasted on high heat until I could smell aroma. They may have burnt a bit. Then added rice/milk mixture. Then added 1 stick canella. It has a tendency to stick to the cup. Then added 1 cup heavy cream. Heated to bare simmer (a little bubble) then removed from heat, covered, and let sit 2 hours</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Strain with wide-option first, then narrow, then cheese cloth</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Milk eggs and sugar</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Make custard</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Add vanilla</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">5th attempt</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Followed serious eats recipe, except chopped almonds and blended rice to try to get ½ size. Added some full rice to get back to ½ cup. Strained out small pieces. added ¼ tsp xanthan gum</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Future options</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Could let the milk and rice soak together longer</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Could use half and half instead of milk/heavy cream</span></li></ul></ul><h3><b>Key Lime Pie</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><a href="http://www.myrecipes.com/recipe/key-lime-pie-ice-cream-50400000113415/" data-mce-href="http://www.myrecipes.com/recipe/key-lime-pie-ice-cream-50400000113415/"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">http://www.myrecipes.com/recipe/key-lime-pie-ice-cream-50400000113415/</span></a></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Changes I made</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Added extra ⅛ of sugar</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Added an extra egg yolk</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Added 4 drops green, 4 drops yellow food coloring</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used graham cracker crumbs mixed with butter instead</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used about 7 key limes</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">A little too tart</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Color was perfect</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Way too hard; had to leave out for 15 minutes</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Texture was icy</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Zest was too big, make it into smaller pieces</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Could make key lime syrup next time to decrease water content</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Could use an alcohol to help with iciness. Tequila</span></li></ul><p><br></p><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><br></li></ul><h3><b>Mexican Chocolate</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">David Lebovitz</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Great flavor</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Smooth texture</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Added a little more cinnamon</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used high range of chile powder</span></li></ul><h3><b>Mexican Chocolate – Queen City Chocolate&nbsp;</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used the recipe from Jenni’s ice cream book</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Doubled the recipe since they usually only made one quart and it had been going fast</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The recipe called for 1/8th teaspoon cayenne (to taste) and ½ teaspoon cinnamon for the normal amount. I used one and a half times the cayenne and double the cinnamon.</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">I don’t know if I churned it for long enough, it was only in for 30 minutes and I took it out before I heard the ice cream maker struggling. I used extra ice and rock salt, though. The consistency was slightly liquidy when I took it out.</span></li></ul><h3><b>Mint Chocolate Chip</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">First batch</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Texture and flavor were great.</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The mint was very earthy because I didn’t use any extract</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Next time I could consider using both mint leaves and extract</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">One recipe mentioned blending the mint with the milk/cream/sugar. Might try that</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">I just chopped the mint</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">When extracting, it was hard to get the last bit of mint out from the leaves. Might try some other method for pressing aside from using the spatula.</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Haven’t found a good chocolate yet. The chocolate chunk seems the most original, but the flavor is so-so. I used semi-sweet in the last batch and they didn’t pair well</span></li></ul></ul><p><a href="http://www.williams-sonoma.com/recipe/mint-chocolate-chip-ice-cream.html" data-mce-href="http://www.williams-sonoma.com/recipe/mint-chocolate-chip-ice-cream.html"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">http://www.williams-sonoma.com/recipe/mint-chocolate-chip-ice-cream.html</span></a></p><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">6/7/18 second batch</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Donna brought fresh mint from her yard. I used the recipe from David Leibovitz. To extract all the flavor from the mint I squeezed it with my hand. When I tasted the liquid I thought the mint flavor was too subtle, but then the ice cream had a great taste. I chopped up chocolate chips (semi-sweet) and put them in the churner just before it finished. Donna said that she really liked his one. Howard and Christine aren’t fans of mint.&nbsp;</span></li></ul></ul><h3><b>Orange Popsicle</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">David Lebovitz recipe</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">processing the zest with the sugar made a delicious sugar, but the grated zest was unpleasant. Want to figure out a way to get rid of the bad texture. Need to remove the zest at some point or pulverize it.</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">the flavor was amazing</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">texture was ok, a little icy. Could have used more liquor</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">used 6 drops of red food coloring to give it a deep orange color</span></li></ul><h3><b>Peach</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Two different recipes. Macerated peaches and blended peaches.&nbsp;</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">First time the peaches weren't ripe enough so the flavor wasn't pronounced and the peaches were tough.&nbsp;</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Second time I let the peaches ripen more and blended them together. Flavor was great. The ice cream was very icy. I tried to reduce the liquid but I put the sugar in and that prevented the reduction to some degree. Could have added more heavy cream.&nbsp;</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">1-16-18: Made the David Lebovitz recipe for Josie’s 100th birthday</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">I was a little short on the number of peaches and I combined frozen peaches with raw peaches.</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The recipe has you cook them and they got burnt. Some were burnt like crazy. I didn’t want to add the water in right away so I waited too long. I was worried since I had the frozen peaches.</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">It ended up tasting very burnt and being very icy</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">1-17-18: Made Lebovitz recipe again for Josie’s 100th birthday</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Got fresh peaches this time. Had a tough time getting the pits out. Let them cook down on low heat for about an hour. Kept on siphoning off the evaporated water. I also doubled the vanilla. The taste was really good as I was putting it in the container</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">6/7/18: made this with creme fraiche for a peaches and cream.&nbsp;</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">This was in the middle of peach season so the peaches were very ripe. There was only a small food processor so I had to break it into portions. I left it more chunky than normal. It was a little icy but overall very good.</span></li></ul></ul><h3><b>Pineapple Sorbet</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">David lebovitz recipe</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Texture was bad; may strain next time</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Flavor was good</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Very icy</span></li></ul><h3><b>Pistachio</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">1st attempt in Beaumont Apt ‘13 or ‘14</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Recipe from http://www.floatingkitchen.net/pistachio-dark-chocolate-chip-ice-cream/</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used 1.5x the almond extract. The flavor was too strong, use less next time.&nbsp;</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Smaller dark chocolate pieces next time</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">This one was pretty good</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">2nd attempt on Dec ‘17</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Recipe from Van Leeuwen book</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Very delicious! Followed recipe exactly</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">3rd attempt Jan ‘19 for Muma’s</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><a href="https://lifemadesimplebakes.com/the-best-homemade-pistachio-ice-cream/" data-mce-href="https://lifemadesimplebakes.com/the-best-homemade-pistachio-ice-cream/"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">https://lifemadesimplebakes.com/the-best-homemade-pistachio-ice-cream/</span></a></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">¾ c. shelled pistachios, divided*</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">½ c. sugar, divided (¼ cup corn syrup, ¼ cup sugar)</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">1½ c. whole milk</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">1½ c. heavy cream</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">3 eggs + 2 yolks</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">¼+ 1/8 tsp. almond extract</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">¼ tsp. vanilla extract or paste</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">¼ teaspoon xanthum gum</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Made own pistachio “paste” as part of the recipe</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Overall was good but ended up being really hard</span></li></ul></ul><p><br></p><h3><b>Pralines and Pecan</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Emeril’s recipe </span><a href="http://www.foodnetwork.com/recipes/emeril-lagasse/pralines-and-cream-ice-cream-recipe.html" data-mce-href="http://www.foodnetwork.com/recipes/emeril-lagasse/pralines-and-cream-ice-cream-recipe.html"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">http://www.foodnetwork.com/recipes/emeril-lagasse/pralines-and-cream-ice-cream-recipe.html</span></a></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Didn’t put the caramel into the ice cream; added it separately with each serving.</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Getting the sauce out each time was a bit of a hassle</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Next time could layer it in at very end of cycle or as I put it in a container</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Pralines were large; next time cut them up and use less (¾ as many)</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used a teaspoon bourbon. Could do it again next time</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Seemed like more than 1.5 quarts of ice cream</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Texture was amazing, pralined pecans had the exact right consistency</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Flavor was amazing, especially with the caramel sauce.&nbsp;</span></li></ul><h3><b>Pumpkin</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">First attempt-Bad</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Did not add enough sugar (didn’t follow the recipe)</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Was not very good</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Second attempt-Good</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Added three tablespoons spiced rum</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><a href="http://www.howsweeteats.com/2013/11/chocolate-chip-cookies-and-cream-pumpkin-marshmallow-ice-cream/" data-mce-href="http://www.howsweeteats.com/2013/11/chocolate-chip-cookies-and-cream-pumpkin-marshmallow-ice-cream/"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Recipe</span></a><span style="font-weight: 400;" data-mce-style="font-weight: 400;">&nbsp;</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Didnt add any cookies</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used homemade pumpkin spice with store-bought ground cloves</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used 2 cups heavy cream, ½ cup whipping cream and 1½ cup milk</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">A little icy. Maybe needs more fat content or faster freezing</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Added a ton of food coloring</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Third attempt-Great</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Added three tablespoons spiced rum</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><a href="http://www.howsweeteats.com/2013/11/chocolate-chip-cookies-and-cream-pumpkin-marshmallow-ice-cream/" data-mce-href="http://www.howsweeteats.com/2013/11/chocolate-chip-cookies-and-cream-pumpkin-marshmallow-ice-cream/"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Recipe</span></a><span style="font-weight: 400;" data-mce-style="font-weight: 400;">&nbsp;</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Didnt add any cookies</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used homemade pumpkin spice with store-bought ground cloves</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used 2 ½ cups heavy cream and 1½ cup milk</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">A little icy (less than attemp #2). Maybe needs more fat content or faster freezing</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used 3 portions of food coloring(orange)</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Fourth attempt&nbsp;</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used 3 cups heavy cream and 1 cup milk. May consider eggs or cornstarch next time</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Fifth attempt</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used 150g + some sugar. It needed more</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used ¼ teaspoon xanthum gum</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">2 cups heavy cream, 1 cup half and half, 1 cup milk, 5 eggs. Hard to tell when the eggs were cooked</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Was still icy! It was a little mealy.</span></li></ul></ul><h3><b>Pumpkin Joe-Joe</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used David Lebovitz base with pinch xanthum gum, 1 tablespoon dry milk powder, and ½ teaspoon of pumpkin pie spice (2% milk instead of whole). Added ¼ teaspoon vanilla at the end</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The custard ended up being very lightly custard iced. Only one like chunk got caught in the strainer</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used ice up to the top and 1 lb ice</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used 10 joe joes</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Second time used 9 joe-joes. 2 tablespoon dry milk powder, 1% milk, ¼ pumpkin pie spice</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><br></li></ul><h3><b>Rhubarb Ice Cream</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Made this on 6-6-19</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Rhubabrb ice cream from NY times. I left out the caramel sauce and cooked the rhubarb a smidge too long.&nbsp;</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">I used a little extra of all the ingredients.&nbsp;</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">I used a teeny bit of xanthum gum and it wasn’t enough. I’d like to use a full amount next time.&nbsp;</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">When I added it to the milk and heated it got gummy so I threw that out and added it to the end on the redo.&nbsp;</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">I also think I should try to fold in the compote in the container instead of in the ice cream machine</span></li></ul><h3><b>S'mores</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><a href="http://www.browneyedbaker.com/2012/06/27/smores-ice-cream/" data-mce-href="http://www.browneyedbaker.com/2012/06/27/smores-ice-cream/"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Internet Recipe</span></a></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Great taste (unsure)</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Good texture (unsure)</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Chocolate sauce was great</span></li></ul><p><br></p><h3><b>Saffron</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Made the David Lebovitz recipe for 1/26/18</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Let the Saffron soak for too long (longer than recipe), the flavor was too bold.</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">There may have been too much sugar too. The sweetness was a little too much</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">I served this with Pistachios</span></li></ul><h3><b>Samoa</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><a href="https://sites.google.com/site/kitchenquestrecipes/samoa-ice-cream" data-mce-href="https://sites.google.com/site/kitchenquestrecipes/samoa-ice-cream"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Internet Recipe</span></a></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Great taste (unsure)</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Good texture (unsure)</span></li></ul><p><br></p><h3><b>Stracciatella</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Made this with the white chocolate ice cream. The texture was perfect, it broke when spoon went through it.</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The flavor could be improved slightly. It tastes just like the chocolate bar that I bought.</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Need to find a way to drizzle this in while the ice cream is churning. Might need a new ice cream machine.</span></li></ul><h3><b>Strawberry</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Pioneer Woman Recipe: </span><a href="http://thepioneerwoman.com/cooking/2013/07/strawberry-ice-cream/" data-mce-href="http://thepioneerwoman.com/cooking/2013/07/strawberry-ice-cream/"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">http://thepioneerwoman.com/cooking/2013/07/strawberry-ice-cream/</span></a></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">A little icy</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used spiced rum and the flavor was a little noticeable. Next time use vodka or similar instead. Maybe amaretto would work</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Could use with some straining of the strawberry seeds. It is not really an issue though.</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The color is from food coloring. Could have used half of what I did and had the same color I think. Using too much food coloring gives it a funny mouth feeling.</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Flavor was great</span></li></ul><h3><b>Strawberry Sour Cream</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Made on 6/7/18</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Recipe from David Leibovitz. The strawberries seemed to be in season and were very fresh</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The color was amazing</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The texture was a little icier since it was fruit based. There were some frozen chunks. Everyone really liked it</span></li></ul><h3><b>Sweet Corn w/ Raspberry sauce</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used the recipe from Jenni’s ice cream book</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Flavor was amazing - a little cheesy at first but then got used to it.</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Tasted like corn, but also a bit like cheesecake</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Need to do a better job layering the sauce and ice cream</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Texture was perfect</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Made this again on 9-21-17</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">This time I doubled the recipe</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">First taste seems a little icy. I think I need to go harder on the salt and ice next time</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Also, when boiling for 1 minute after the corn starch addition, I let the bottom burn so there were black specs sifting around. Next time I need to be sure to stir constantly while I have it on med-high heat</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The doubling was way too much. And when the ice cream froze, the bottom and walls got more icy than the rest. The ice cream was icy overall. Taste was good.</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The raspberry sauce was way too jelly</span></li></ul></ul><h3><b>Thai Chai</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Made this on 8/16/19</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">https://www.loveandoliveoil.com/2017/05/thai-tea-ice-cream.html</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">https://www.thespruceeats.com/real-thai-iced-tea-recipe-765458</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Hybrid recipe of Thai tea (using Ceylon and not Thai Tea mix) and Chai</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Ingredients</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">2 star anise</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">3 packets of Ceylon black tea</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">8 cloves</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">2 cardamom pods</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">1/2 stick canella cinnamon</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">1 can condensed milk</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">2 cups 2% milk</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">1 cup heavy cream (plus a little extra)</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Pinch salt</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">1/4 teaspoon xantham gum (added while making custard)</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">When churning I added a ton of ice and shook the machine a few times to get it to transfer heat better</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">It ended up being really hard. Maybe this is due to the xantham gum?</span></li></ul><h3><b>Toasted Coconut</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Flavor was amazing</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Texture was amazing</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used 1/2 teaspoon vanilla and 1/2 teaspoon of rum</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">David Lebovitz recipe</span></li></ul><h3><b>Vanilla</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Alton Brown’s Recipe</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Easy to remember recipe</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><br></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">David Lebovitz’ Recipe</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Amazing texture</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Amazing flavor</span></li></ul></ul><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><br></li></ul><h3><b>Vegan Chocolate&nbsp;</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Recipe from Van Levyn book</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">I was short on the chocolate by about 25-50% so I replaced with Coco powder</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Texture was a little like astronaut ice cream</span></li></ul><h3><b>Watermelon Sherbet</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Recipe: </span><a href="http://allrecipes.com/recipe/watermelon-sherbet/" data-mce-href="http://allrecipes.com/recipe/watermelon-sherbet/"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">http://allrecipes.com/recipe/watermelon-sherbet/</span></a></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The texture was a little icy—completely different than other ice creams I have made</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">It was very light.</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">It wasn’t noticeable ice crystals.</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The flavor was subtle watermelon. Definitely had a similarity to the Jolly Rancher watermelon flavor, but it wasn’t appalling to me like the candy is.&nbsp;</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">I think I could have improved this with some other additional flavors, like orange or lime.</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">The texture could have been improved with more cream I think.</span></li></ul><h3><b>White Chocolate</b></h3><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">David Lebovitz Recipe</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Following recipe exactly</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Was extremely creamy</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Perfect texture</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Really rich</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Made it with the fudge sauce that wasn’t as good</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Would be better to do a harder chocolate next time</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Would go well with fruit swirl or sauce (raspberry or blackberry, or strawberry (balsamic?))</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Made again christmas 2014. Perfect texture both times&nbsp;</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Made again for superbowl 2015</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used only 4 eggs and texture was still perfect</span></li></ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Made as second ice cream in Atlanta for group 15 meeting</span></li><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Used half-and-half instead of whole milk. Borderline overly rich</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><br></li></ul></ul><h3><b>White Chocolate and Ancho Chile</b></h3><p><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Recipe: http://www.fieryfoodscentral.com/recipe/white-chocolate-ancho-chile-ice-cream/</span></p><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">3 ancho chiles, stems removed</span></li></ul><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">½ teaspoon ground cinnamon</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">1/4 teaspoon ground cloves</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">6 ounces (2 bars) good quality white chocolate such as Tobler or Lindt</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">2 cups heavy cream</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">2 cups milk</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">3/4 cup sugar</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">1 vanilla bean</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">6 egg yolks</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Cinnamon stick and shaved dark semisweet chocolate for garnish</span></li></ul><p><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Quality</span></p><ul><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">Great flavor. Kind of like pumpkin because of the cinnamon and cloves</span></li><li style="font-weight: 400;" data-mce-style="font-weight: 400;"><span style="font-weight: 400;" data-mce-style="font-weight: 400;">I wish Ancho chile chunks were smaller and the ancho flavor was more subtle.</span></li></ul></body>"""
soup = bs4.BeautifulSoup(ice_cream_page, 'html.parser')

In [ ]:
# Walk through the list of nodes in bs4, add text to a storage
ice_creams = []; item = []; title = ""; text = ""

for child in soup.recursiveChildGenerator():
    level = len(child.find_parents('ul'))
    if child.name in ["h3"]:
        if title:
            ice_creams.append([title.strip(), text])
            text = ""
            title = ""
        title = child.text.strip()+"\n"
    if child.name in ["b", "span"]:
        if child.find_parents('h3'): continue
        if level > 0: 
            text += "\t" * (level-1) + "* "
        text += child.text + '\n'

#### Movies

In [ ]:
movies=[
["Rushmore", "Wes Anderson’s first film"],
["Jaws", "Classic that I haven't seen"],
["The Last of the Mohicans", "Have heard a lot about this movie"],
["Pi", "psychological thriller by Darren Aronofsky"],
["The Big Sick", "Christine has already watched—it is good"],
["A Few Good Men", "Howard really likes this movie and thinks there are amazing scenes"],
["The Interview", "North Korea made a big deal about this movie"],
["Footloose","Christine has seen. This is a classic"],
["Jerry McGuire", "Howard said that he wanted to watch this with me"],
["Return of the Cat", "Christine wants to watch this with me. It is a Takahata movie"],
["The Fighter", "I love Darren Aronofsky"],
["Walk the Line", "Johnny Cash movie that was really popular"],
["Spinal Tap", "My mom talks about this movie a lot. Donna and Howard want to watch it"]
]

### Import Data into Notion


In [ ]:
import mistletoe
from md2notion.upload import upload, uploadBlock
from md2notion.NotionPyRenderer import NotionPyRenderer
from notion.block import PageBlock

# Load to test page to verify and then move over
gtd_table = client.get_collection_view("https://www.notion.so/declannishiyama/e93479a9604b4f08923f3efb80afa640?v=e285d928e4194aa7bae9943894704977")

items = movies
# [0] is title, [1] is the content

for idx, item in enumerate(items):
    print(f"\rUploading {item[0]}, {idx+1}/{len(items)}", end='\t\t\t\t\t')
    
    newRow = gtd_table.collection.add_row(title=item[0])
    rendered = mistletoe.markdown(item[1], NotionPyRenderer)
    
    for idx, blockDescriptor in enumerate(rendered):
        uploadBlock(blockDescriptor, newRow, None)
              

### Calendar

#### Filtering

In [ ]:
filter_params = [{
    "property": "context",
    "comparator": "contains",
    "value": "GTD",
}]

In [ ]:
calendar_url = "https://www.notion.so/declannishiyama/dec049b9497f4b269a33220b541f079b?v=10076f8bdf1942b1b59b0f5d19a2ae43"
calendar = client.get_collection_view(calendar_url)
calendar_entries = calendar.build_query(filter=filter_params).execute()
# calendar_entries = calendar.build_query().execute()
# collection = calendar.collection
# schema = collection.get_schema_properties()
len(calendar_entries)

#### Timezone correction

In [ ]:
import json, os, pytz, logging
from datetime import datetime, date, timezone, timedelta
from icalendar import Calendar, Event
from notion.client import NotionClient
from notion.collection import CalendarView
from notion.block import BasicBlock
from notion.user import User

In [ ]:
calendar_url = "https://www.notion.so/declannishiyama/dec049b9497f4b269a33220b541f079b?v=07e4a7faadbc49068317418db1b2f9cb"
calendar = client.get_collection_view(calendar_url)
calendar_query = calendar.build_query()
calendar_entries = calendar_query.execute()
len(calendar_entries)

In [ ]:
properties_by_name = {}
properties_by_slug = {}
properties_by_id = {}
title_prop = None

collection = calendar.collection
schema = collection.get_schema_properties()

for prop in schema:
    name = prop['name']
    if name in properties_by_name:
        print("WARNING: duplicate property with name {}".format(name))
    properties_by_name[name] = prop
    properties_by_slug[prop['slug']] = prop
    properties_by_id[prop['id']] = prop
    if prop['type'] == 'title':
        title_prop = prop

assert title_prop is not None, "Couldn't find a title property"

dateprop = properties_by_id[calendar_query.calendar_by]; dateprop

In [ ]:
[[i, e2]] = [[i, c] for i, c in enumerate(calendar_entries) if "Review Studiomates interview" in c.__repr__()]; i,e

In [ ]:
[[i, e1]] = [[i, c] for i, c in enumerate(calendar_entries) if "Call IRS" in c.__repr__()]

In [ ]:
e_date = e1.get_property(dateprop['id']); e_date.start

In [ ]:
# e_date.start.astimezone(pytz.timezone('US/Eastern')).astimezone(timezone.utc)
e_date.start + timedelta(hours=5)

In [ ]:
e_date.start.astimezone(pytz.timezone('US/Eastern'))

In [ ]:
e_date = e.get_property(dateprop['id']); 

In [ ]:
ical = get_ical(client, calendar_url, "{NAME}")

In [ ]:
date.start.astimezone(pytz.timezone('US/Eastern'))
pytz.timezone('US/Eastern').localize(date.start)

In [ ]:
date.start

In [ ]:
ical.to_ical()

In [ ]:
def get_ical(client, calendar_url, title_format):
    calendar = client.get_collection_view(calendar_url)
    if not isinstance(calendar, CalendarView): 
        raise Exception(f"Couldn't find a calendar view in the following list: {calendar.views}")

    calendar_query = calendar.build_query()
    calendar_entries = calendar_query.execute()
    logging.info(f'Found {len(calendar_entries)} calendar entries')

    collection = calendar.collection

    schema = collection.get_schema_properties()

    properties_by_name = {}
    properties_by_slug = {}
    properties_by_id = {}
    title_prop = None

    for prop in schema:
        name = prop['name']
        if name in properties_by_name:
            print("WARNING: duplicate property with name {}".format(name))
        properties_by_name[name] = prop
        properties_by_slug[prop['slug']] = prop
        properties_by_id[prop['id']] = prop
        if prop['type'] == 'title':
            title_prop = prop

    assert title_prop is not None, "Couldn't find a title property"

    dateprop = properties_by_id[calendar_query.calendar_by]
    #assert dateprop['type'] == 'date', "Property '{}' is not a Date property".format(settings['property'])

    cal = Calendar()
    cal.add("summary", "Imported from Notion, via notion-export-ics.")
    cal.add('version', '2.0')
    
    #### START LAST SYNC INFO ####
    last_synced = Event()
    last_synced.add('dtstart', datetime.now(tz=pytz.timezone('US/Eastern')).date())
    last_synced.add('summary', "Last Sync")
    last_synced.add('description', datetime.now(tz=pytz.timezone('US/Eastern')).strftime('%H:%M:%S'))
    cal.add_component(last_synced)
    #### END LAST SYNC INFO ####

    for e in calendar_entries:
        if e.get_property('status') == 'Done': continue
    
        e_date = e.get_property(dateprop['id'])
        if e_date is None: continue

        name = e.get_property(title_prop['id'])
        clean_props = {'NAME': name}

        # Put in ICS file
        event = Event()
        desc = ''
        
        # Add timezone if possible
        if type(e_date.start) == datetime:
            event.add('dtstart', e_date.start.astimezone(pytz.timezone('US/Eastern')).astimezone(timezone.utc))
        else: # otherwise it is probably a date
            event.add('dtstart', e_date.start)
        
        if type(e_date.end) == datetime:
            event.add('dtend', e_date.end.astimezone(pytz.timezone('US/Eastern')).astimezone(timezone.utc))
        elif type(e_date.end) == date:
            event.add('dtend', e_date.end)
        if e_date.end is None:
            pass
                
        desc += e.get_browseable_url() + '\n\n'
        desc += 'Properties:\n'
        for k, v in e.get_all_properties().items():
            if k != dateprop['slug']:
                name = properties_by_slug[k]['name']
                desc += "  - {}: {}\n".format(name, v)
                clean_props[name] = v
        title = title_format.format_map(clean_props)
        event.add('summary', title)
        event.add('description', desc)
        cal.add_component(event)

    logging.info(f'Calendar has {len(cal.subcomponents)} entries')        

    return cal

In [ ]:
Out[123][1].get_property(date)

In [ ]:
cal = Calendar()

In [ ]:
cal.add("summary", "Imported from Notion, via notion-export-ics.")
cal.add('version', '2.0')
    
    #### START LAST SYNC INFO ####
last_synced = Event()
last_synced.add('dtstart', datetime.now(tz=pytz.timezone('US/Eastern')).date())
last_synced.add('summary', "Last Sync")
last_synced.add('description', datetime.now(tz=pytz.timezone('US/Eastern')).strftime('%H:%M:%S'))
cal.add_component(last_synced)

In [ ]:
len(cal.subcomponents)

In [ ]:
len(cal), cal.items()

In [ ]:
collection.get_schema_properties()

In [ ]:
len(calendar_entries); #341

In [ ]:
Out[53].context

In [ ]:
import pytz

In [ ]:
notion_ics = __import__("notion-export-ics.notion_ics")
# notion_export_ics = __import__("notion-export-ics")
notion_ics.notion_ics.get_ical

In [ ]:
datetime.now(tz=pytz.timezone('US/Eastern')).strftime('%H')

In [ ]:
ical = notion_ics.notion_ics.get_ical(
    client, 
    "https://www.notion.so/declannishiyama/dec049b9497f4b269a33220b541f079b?v=10076f8bdf1942b1b59b0f5d19a2ae43", 
    "{NAME}").to_ical()
print(ical)

In [ ]:
calendar_url = "https://www.notion.so/declannishiyama/dec049b9497f4b269a33220b541f079b?v=10076f8bdf1942b1b59b0f5d19a2ae43"
calendar = client.get_collection_view(calendar_url)
calendar_query = calendar.build_query()
calendar_entries = calendar_query.execute()
collection = calendar.collection
schema = collection.get_schema_properties()

properties_by_name = {}
properties_by_slug = {}
properties_by_id = {}
title_prop = None

for prop in schema:
    name = prop['name']
    if name in properties_by_name:
        print("WARNING: duplicate property with name {}".format(name))
    properties_by_name[name] = prop
    properties_by_slug[prop['slug']] = prop
    properties_by_id[prop['id']] = prop
    if prop['type'] == 'title':
        title_prop = prop

assert title_prop is not None, "Couldn't find a title property"

dateprop = properties_by_id[calendar_query.calendar_by]
#assert dateprop['type'] == 'date', "Property '{}' is not a Date property".format(settings['property'])

cal = Calendar()
cal.add("summary", "Imported from Notion, via notion-export-ics.")
cal.add('version', '2.0')

# Add last synced information #####
last_synced = Event()
last_synced.add('dtstart', datetime.today().date())
last_synced.add('summary', "Last Sync")
last_synced.add('description', datetime.today().strftime('%H:%M:%S'))
cal.add_component(last_synced)

for e in calendar_entries:
    date = e.get_property(dateprop['id'])
    if date is None:
        continue

    name = e.get_property(title_prop['id'])
    clean_props = {'NAME': name}

    # Put in ICS file
    event = Event()
    desc = ''
    event.add('dtstart', date.start)
    if date.end is not None:
        event.add('dtend', date.end)
    desc += e.get_browseable_url() + '\n\n'
    desc += 'Properties:\n'
    for k, v in e.get_all_properties().items():
        if k != dateprop['slug']:
            name = properties_by_slug[k]['name']
            desc += "  - {}: {}\n".format(name, v)
            clean_props[name] = v
    title = name
    event.add('summary', title)
    event.add('description', desc)
    cal.add_component(event)

print(cal.to_ical())

In [ ]:
with open('test.ics', 'w') as filehandle:
    for line in cal.content_lines():
        filehandle.write("%s\n" % line)

In [ ]:
datetime.today().strftime('%H:%M:%S')

In [ ]:
date.start

In [ ]:
# cal.content_lines()
date = calendar_entries[-200].get_property(dateprop['id'])

In [ ]:
date.start

In [ ]:
import json
from datetime import datetime
from icalendar import Calendar, Event
from notion.client import NotionClient
from notion.collection import CalendarView
from notion.block import BasicBlock
from notion.user import User

# Hack some representation stuff into notion-py

BasicBlock.__repr__ = BasicBlock.__str__ = lambda self: self.title
User.__repr__ = User.__str__ = lambda self: self.given_name or self.family_name


def get_ical(client, calendar_url, title_format):
    calendar = client.get_collection_view(calendar_url)
    if not isinstance(calendar, CalendarView): 
        raise Exception(f"Couldn't find a calendar view in the following list: {calendar.views}")

    calendar_query = calendar.build_query()
    calendar_entries = calendar_query.execute()

    collection = calendar.collection

    schema = collection.get_schema_properties()

    properties_by_name = {}
    properties_by_slug = {}
    properties_by_id = {}
    title_prop = None

    for prop in schema:
        name = prop['name']
        if name in properties_by_name:
            print("WARNING: duplicate property with name {}".format(name))
        properties_by_name[name] = prop
        properties_by_slug[prop['slug']] = prop
        properties_by_id[prop['id']] = prop
        if prop['type'] == 'title':
            title_prop = prop

    assert title_prop is not None, "Couldn't find a title property"

    dateprop = properties_by_id[calendar_query.calendar_by]
    #assert dateprop['type'] == 'date', "Property '{}' is not a Date property".format(settings['property'])

    cal = Calendar()
    cal.add("summary", "Imported from Notion, via notion-export-ics.")
    cal.add('version', '2.0')

    for e in calendar_entries:
        date = e.get_property(dateprop['id'])
        if date is None:
            continue

        name = e.get_property(title_prop['id'])
        clean_props = {'NAME': name}

        # Put in ICS file
        event = Event()
        desc = ''
        event.add('dtstart', date.start)
        if date.end is not None:
            event.add('dtend', date.end)
        desc += e.get_browseable_url() + '\n\n'
        desc += 'Properties:\n'
        for k, v in e.get_all_properties().items():
            if k != dateprop['slug']:
                name = properties_by_slug[k]['name']
                desc += "  - {}: {}\n".format(name, v)
                clean_props[name] = v
        title = title_format.format_map(clean_props)
        event.add('summary', title)
        event.add('description', desc)
        cal.add_component(event)

        # Print
        #print("{}: {} -> {}".format(title, date.start, date.end))
        #print(desc)
        #print('--------------')

    return cal

In [ ]:
ical = get_ical(
    client, 
    "https://www.notion.so/declannishiyama/dec049b9497f4b269a33220b541f079b?v=10076f8bdf1942b1b59b0f5d19a2ae43", 
    "{NAME}").to_ical()
print(ical)

In [ ]:
# client.get_record_data("collection_view", '49edd0f7-7c48-4aa8-a9b7-15ed0d19a3cc')
# {'id': '49edd0f7-7c48-4aa8-a9b7-15ed0d19a3cc',
#  'version': 3,
#  'type': 'calendar',
#  'name': 'Calendar View',
#  'parent_id': '6eb2e42b-3f65-4081-8cdd-d3c7df444ebf',
#  'parent_table': 'block',
#  'alive': True,
#  'query2': {'calendar_by': '*f$t'}}

In [ ]:
client.get_collection_view("https://www.notion.so/declannishiyama/6eb2e42b3f6540818cddd3c7df444ebf?v=90c4436d0a76405088a842a8e95e60e3")

In [ ]:
Out[82].get_schema_properties()

### Get Might Could Style image information

In [2]:
original_path  = Path('/Users/nish/Downloads/original_artwork'); original_path
if not os.path.exists(original_path):
    os.makedirs(original_path)
        
def save_image(filename, url):
    img_data = requests.get(url).content
    with open(original_path/filename, 'wb') as handler:
        handler.write(img_data)
            
# Then use photoshop to convert to JPG and the correct size

In [3]:
# artwork_url='https://www.notion.so/mightcould/abbdeac84d354adf808dc4baa562f27d?v=4a531fc433c349ffa570a0c519c566ba'
artwork_url = 'https://www.notion.so/mightcould/a01fb28d83b14e34af1af0f1ae8acc42?v=e0a6079e38a44d2780eccd20f903e78b'
artwork_view = client.get_collection_view(artwork_url)
art_rows = artwork_view.collection.get_rows(); len(art_rows)

30

In [58]:
# 60s 
existing_filenames = [g.split('/')[-1] for g in glob.glob(str(original_path/'*'))]; existing_filenames[0]

artwork = {}
question_dict = {}
for i, art_row in enumerate(art_rows):
#     if not art_row.get_all_properties()['question'] == 'Cats': continue
    print(f'\r {i}/{len(art_rows)}', end='')
    img_url = art_row.children[0].source
    filename = img_url.split('/')[5].split('?')[0]
    if '%28' in filename or '%29' in filename: # Does this causes problems
        filename = filename.replace('%28','(').replace('%29',')')

    if filename not in existing_filenames:
        print(f'Saving image {filename}')
        save_image(filename, img_url)

    art_props = art_row.get_all_properties(); art_props
    art_props = { **art_props, 'space': re.sub(',.+', '', art_props['space']) } # fix space issue
    art_details = {
        'categories':{f'{k}>{v}':1 for k,v in art_props.items() if k not in ['name', 'question']},
        'question': art_props['question'],
        'name': art_props['name'],
    }
    jpg_filename = '.'.join([filename.rsplit('.', 1)[0], 'jpg'])
    artwork[jpg_filename] = art_details
    question_dict.setdefault(art_details['question'],[]).append(jpg_filename)
    
# THEN USE PHOTOSHOP TO RESIZE THE IMAGES

 29/30

In [59]:
# Get the list of features
print('export const features=[')
for p in artwork_view.collection.get_schema_properties():
    if p['slug'] in ['name', 'question']: continue
    print(f'\t"{p["slug"]}",')
print(']')

export const features=[
	"shape_form_defined_by",
	"level_of_realism",
	"color_lightness",
	"line_width",
	"texture",
	"color_saturation",
	"space",
	"line_quality",
]


In [60]:
# Get the list of artwork for copying
print('export const options={')
for f,d in artwork.items():
    print(f'\t"{f}":{d},')
print('}')

const options={
	"Utagawa_Kuniyoshi_Cats_Suggested_as_the_Fifty-three_Stations_of_the_Tokaido_1850.jpg":{'categories': {'shape_form_defined_by>Line': 1, 'level_of_realism>Abstracted Realism': 1, 'color_lightness>Lighter': 1, 'line_width>Thin': 1, 'texture>Smooth strokes': 1, 'color_saturation>Duller': 1, 'space>2-D Pictorial Plane': 1, 'line_quality>Consistent': 1}, 'question': 3, 'name': 'Utagawa Kuniyoshi: Cats Suggested as the Fifty-three Stations of the Tōkaidō, 1850'},
	"Theophile_Steinlen_The_Chat_Noir_1896.jpg":{'categories': {'shape_form_defined_by>Line': 1, 'level_of_realism>Abstracted Realism': 1, 'color_lightness>Darker': 1, 'line_width>Thick': 1, 'texture>Smooth strokes': 1, 'color_saturation>Brighter': 1, 'space>2-D Pictorial Plane': 1, 'line_quality>Varied': 1}, 'question': 4, 'name': 'Théophile Steinlen: The Chat Noir, 1896'},
	"Henriette_Ronner-Knip_The_cat_at_play_c._1860__1878.jpg":{'categories': {'shape_form_defined_by>Value': 1, 'level_of_realism>Realism': 1, 'color

In [56]:
# Get the question_dict for copying
# print('const questions_dict=') # question_dict
# print(json.dumps(question_dict, indent=2))
questions_list = [{'options':v, 'label':k, 'number':i+1} for i,(k,v) in enumerate(question_dict.items())]
print('const questions_list=')
print(json.dumps(questions_list, indent=2))

const questions_list=
[
  {
    "options": [
      "Utagawa_Kuniyoshi_Cats_Suggested_as_the_Fifty-three_Stations_of_the_Tokaido_1850.jpg",
      "Henriette_Ronner-Knip_The_cat_at_play_c._1860__1878.jpg",
      "Suzanne_Valadon_Raminou_1920.jpg",
      "Marc_Chagall_La_Poete_194950.jpg",
      "Mitsou-Blathus.jpg"
    ],
    "label": 3,
    "number": 1
  },
  {
    "options": [
      "Theophile_Steinlen_The_Chat_Noir_1896.jpg",
      "Pierre_Auguste_Renoir_(1841-1919)_Julie_Manet_also_known_as_Child_with_cat_1887.jpg",
      "_25_Cats_Name_Sam_and_One_Blue_Pussy_-_1954_-_3.jpg",
      "Sleeping_Cat_Pierre-Auguste_Renoir_Date-_1862.jpg",
      "henri-matisse-girl-on-red-couch-with-cat-1938.jpg"
    ],
    "label": 4,
    "number": 2
  },
  {
    "options": [
      "Cats_in_Art-_Pierre_Bonnard_The_White_Cat_1984_Musee_dOrsay_Paris_France..jpg",
      "25_Cats_Name_Sam_and_One_Blue_Pussy_-_Andy_Warhol_-_1954.jpg",
      "LazinessDate-_1896__Artist-_Felix_Edouard_Vallotton.jpg",
      "gu-y

In [ ]:
print('const questions={')

question_dict).reduce((q_accu, q_name, q_idx) => [
  ...q_accu,
  {
    number: q_idx + 1,
    label: `${q_idx + 1}. ${q_name}`,
    options: question_dict[q_name].reduce( (o_accu, o_name, o_idx) => ({
      ...o_accu,
      [numberToLetter(o_idx + 1)]: { //the letter option
        content: o_name,
        categories: Object.keys(artwork[o_name]).filter(p => !['name', 'question'].includes(p)).reduce((c_accu, c_name) => ({
          ...c_accu,
          [`${c_name}>${artwork[o_name][c_name]}`]: 1,
        }), {}),
      }
    }), {} ),
    max_selections: 2,
    default: false,
  }
], [])

In [31]:
# Get all the categories
fix = lambda x: x.upper().replace(': ','__').replace('-','_').replace(' ', '_').replace('+','_')

print('export const CATEGORIES=[')
for prop in artwork_view.collection.get_schema_properties():
    if prop['slug'] in ['name', 'question']: continue
    for option in prop['options']:
        p_name = fix(prop['slug'])
        o_name = fix(option['value'])
        print(f'\t{p_name}__{o_name},')
print(']')


export const CATEGORIES=[
	SHAPE_FORM_DEFINED_BY__LINE,
	SHAPE_FORM_DEFINED_BY__VALUE,
	SHAPE_FORM_DEFINED_BY__COLOR,
	SHAPE_FORM_DEFINED_BY__TEXTURE,
	LEVEL_OF_REALISM__REALISM,
	LEVEL_OF_REALISM__STYLIZED_REALISM,
	LEVEL_OF_REALISM__ABSTRACTED_REALISM,
	LEVEL_OF_REALISM__STYLIZED_REPRESENTATION,
	LEVEL_OF_REALISM__ABSTRACTION,
	COLOR_LIGHTNESS__LIGHTER,
	COLOR_LIGHTNESS__DARKER,
	COLOR_LIGHTNESS__NO_COLOR,
	LINE_WIDTH__THICK,
	LINE_WIDTH__THIN,
	LINE_WIDTH__NO_LINE,
	TEXTURE__VISIBLE_STROKES,
	TEXTURE__SMOOTH_STROKES,
	TEXTURE__NO_STROKES,
	COLOR_SATURATION__BRIGHTER,
	COLOR_SATURATION__DULLER,
	COLOR_SATURATION__NO_COLOR,
	SPACE__2_D_PICTORIAL_PLANE,
	SPACE__3_D_PICTORIAL_PLANE,
	LINE_QUALITY__CONSISTENT,
	LINE_QUALITY__VARIED,
	LINE_QUALITY__BROKEN,
	LINE_QUALITY__NO_LINE,
]


In [32]:
# Export all the categories
fix = lambda x: x.upper().replace(': ','__').replace('-','_').replace(' ', '_').replace('+','_')
for prop in artwork_view.collection.get_schema_properties():
    if prop['slug'] in ['name', 'question']: continue
    for option in prop['options']:
        p_name = fix(prop['slug'])
        o_name = fix(option['value'])
        print(f'export const {p_name}__{o_name}="{prop["slug"]}>{option["value"]}"')

export const SHAPE_FORM_DEFINED_BY__LINE="shape_form_defined_by>Line"
export const SHAPE_FORM_DEFINED_BY__VALUE="shape_form_defined_by>Value"
export const SHAPE_FORM_DEFINED_BY__COLOR="shape_form_defined_by>Color"
export const SHAPE_FORM_DEFINED_BY__TEXTURE="shape_form_defined_by>Texture"
export const LEVEL_OF_REALISM__REALISM="level_of_realism>Realism"
export const LEVEL_OF_REALISM__STYLIZED_REALISM="level_of_realism>Stylized Realism"
export const LEVEL_OF_REALISM__ABSTRACTED_REALISM="level_of_realism>Abstracted Realism"
export const LEVEL_OF_REALISM__STYLIZED_REPRESENTATION="level_of_realism>Stylized Representation"
export const LEVEL_OF_REALISM__ABSTRACTION="level_of_realism>Abstraction"
export const COLOR_LIGHTNESS__LIGHTER="color_lightness>Lighter"
export const COLOR_LIGHTNESS__DARKER="color_lightness>Darker"
export const COLOR_LIGHTNESS__NO_COLOR="color_lightness>No color"
export const LINE_WIDTH__THICK="line_width>Thick"
export const LINE_WIDTH__THIN="line_width>Thin"
export con

In [57]:
# Get all categories for import in functions.js
fix = lambda x: x.upper().replace(': ','__').replace('-','_').replace(' ', '_').replace('+','_')
categories_functionsjs = [] 
for prop in artwork_view.collection.get_schema_properties():
    if prop['slug'] in ['name', 'question']: continue
    for option in prop['options']:
        p_name = fix(prop['slug'])
        o_name = fix(option['value'])
        categories_functionsjs.append(f'{p_name}__{o_name}')
        
print('import { questions_list, ', end='')
for c in categories_functionsjs:
    print(c, end=', ')
print('} from \'./questions\'')

import { questions_list, SHAPE_FORM_DEFINED_BY__LINE, SHAPE_FORM_DEFINED_BY__VALUE, SHAPE_FORM_DEFINED_BY__COLOR, SHAPE_FORM_DEFINED_BY__TEXTURE, LEVEL_OF_REALISM__REALISM, LEVEL_OF_REALISM__STYLIZED_REALISM, LEVEL_OF_REALISM__ABSTRACTED_REALISM, LEVEL_OF_REALISM__STYLIZED_REPRESENTATION, LEVEL_OF_REALISM__ABSTRACTION, COLOR_LIGHTNESS__LIGHTER, COLOR_LIGHTNESS__DARKER, COLOR_LIGHTNESS__NO_COLOR, LINE_WIDTH__THICK, LINE_WIDTH__THIN, LINE_WIDTH__NO_LINE, TEXTURE__VISIBLE_STROKES, TEXTURE__SMOOTH_STROKES, TEXTURE__NO_STROKES, COLOR_SATURATION__BRIGHTER, COLOR_SATURATION__DULLER, COLOR_SATURATION__NO_COLOR, SPACE__2_D_PICTORIAL_PLANE, SPACE__3_D_PICTORIAL_PLANE, LINE_QUALITY__CONSISTENT, LINE_QUALITY__VARIED, LINE_QUALITY__BROKEN, LINE_QUALITY__NO_LINE, } from './questions'


In [34]:
print('const EMPTY_STYLE_VALUES = {')
for c in categories_functionsjs:
    print(f'\t[{c}]: 0,')
print('}')


const EMPTY_STYLE_VALUES = {
	[SHAPE_FORM_DEFINED_BY__LINE]: 0,
	[SHAPE_FORM_DEFINED_BY__VALUE]: 0,
	[SHAPE_FORM_DEFINED_BY__COLOR]: 0,
	[SHAPE_FORM_DEFINED_BY__TEXTURE]: 0,
	[LEVEL_OF_REALISM__REALISM]: 0,
	[LEVEL_OF_REALISM__STYLIZED_REALISM]: 0,
	[LEVEL_OF_REALISM__ABSTRACTED_REALISM]: 0,
	[LEVEL_OF_REALISM__STYLIZED_REPRESENTATION]: 0,
	[LEVEL_OF_REALISM__ABSTRACTION]: 0,
	[COLOR_LIGHTNESS__LIGHTER]: 0,
	[COLOR_LIGHTNESS__DARKER]: 0,
	[COLOR_LIGHTNESS__NO_COLOR]: 0,
	[LINE_WIDTH__THICK]: 0,
	[LINE_WIDTH__THIN]: 0,
	[LINE_WIDTH__NO_LINE]: 0,
	[TEXTURE__VISIBLE_STROKES]: 0,
	[TEXTURE__SMOOTH_STROKES]: 0,
	[TEXTURE__NO_STROKES]: 0,
	[COLOR_SATURATION__BRIGHTER]: 0,
	[COLOR_SATURATION__DULLER]: 0,
	[COLOR_SATURATION__NO_COLOR]: 0,
	[SPACE__2_D_PICTORIAL_PLANE]: 0,
	[SPACE__3_D_PICTORIAL_PLANE]: 0,
	[LINE_QUALITY__CONSISTENT]: 0,
	[LINE_QUALITY__VARIED]: 0,
	[LINE_QUALITY__BROKEN]: 0,
	[LINE_QUALITY__NO_LINE]: 0,
}


In [3]:
# artwork_url='https://www.notion.so/mightcould/abbdeac84d354adf808dc4baa562f27d?v=4a531fc433c349ffa570a0c519c566ba'
artwork_url = 'https://www.notion.so/mightcould/a01fb28d83b14e34af1af0f1ae8acc42?v=e0a6079e38a44d2780eccd20f903e78b'
artwork_view = client.get_collection_view(artwork_url)
art_rows = artwork_view.collection.get_rows(); len(art_rows)

30

In [58]:
# 60s 
existing_filenames = [g.split('/')[-1] for g in glob.glob(str(original_path/'*'))]; existing_filenames[0]

artwork = {}
question_dict = {}
for i, art_row in enumerate(art_rows):
#     if not art_row.get_all_properties()['question'] == 'Cats': continue
    print(f'\r {i}/{len(art_rows)}', end='')
    img_url = art_row.children[0].source
    filename = img_url.split('/')[5].split('?')[0]
    if '%28' in filename or '%29' in filename: # Does this causes problems
        filename = filename.replace('%28','(').replace('%29',')')

    if filename not in existing_filenames:
        print(f'Saving image {filename}')
        save_image(filename, img_url)

    art_props = art_row.get_all_properties(); art_props
    art_props = { **art_props, 'space': re.sub(',.+', '', art_props['space']) } # fix space issue
    art_details = {
        'categories':{f'{k}>{v}':1 for k,v in art_props.items() if k not in ['name', 'question']},
        'question': art_props['question'],
        'name': art_props['name'],
    }
    jpg_filename = '.'.join([filename.rsplit('.', 1)[0], 'jpg'])
    artwork[jpg_filename] = art_details
    question_dict.setdefault(art_details['question'],[]).append(jpg_filename)
    
# THEN USE PHOTOSHOP TO RESIZE THE IMAGES

 29/30

### GTD

In [113]:
# [[i,a] for i,a in enumerate(art_rows)]
a = art_rows[11] 
b = a
a.

<CollectionRowBlock (id='5944524e-8c40-4f41-9eff-ba174c68d904', title='* Pay Employees')>

In [125]:
import datetime

In [130]:
datetime.date.today() >= Out[115].start

True

In [115]:
Out[113].get_all_properties()['start']
Out[115].start

datetime.date(2020, 11, 24)

In [107]:
artwork_url = 'https://www.notion.so/declannishiyama/d58395825d5946be92328246bd827c41?v=4b4820b0c3de4cd5ad77f5a112b961fc'
artwork_view = client.get_collection_view(artwork_url)
art_rows = artwork_view.collection.get_rows(); len(art_rows)

282

In [77]:
properties = art_rows[200].schema; properties
# art_rows[0].collection.get_schema_properties()
# art_rows[0].get_property('available_action')
# if properties['status'] == 'Today'

[{'id': '#e[^', 'slug': 'due', 'name': 'Due', 'type': 'date'},
 {'id': '31!t',
  'slug': 'context',
  'name': 'Context',
  'type': 'multi_select',
  'options': [{'id': 'd3df7d1b-08aa-41c4-8697-b8cb92fea735',
    'color': 'gray',
    'value': 'None'},
   {'id': 'a72b64e7-7094-4167-aff6-347ca37729f7',
    'color': 'yellow',
    'value': 'Plan'},
   {'id': '785422c7-1a99-43ce-9135-31f17d42141b',
    'color': 'purple',
    'value': '#Avoiding'},
   {'id': '01ed6609-0953-43f1-9841-d20b3feb3dff',
    'color': 'brown',
    'value': '#Relaxing Fun'},
   {'id': 'd8a92dc9-cd14-4d6b-a96c-b175a98709c3',
    'color': 'orange',
    'value': '#Relaxing Work'},
   {'id': '64406b19-0dc8-4a17-82b2-80c1c83ebb81',
    'color': 'yellow',
    'value': '#Motivation'},
   {'id': '95fdd3ad-1796-4e18-97be-70ab5faa2918',
    'color': 'green',
    'value': '#Life Task'},
   {'id': 'bf5c26a5-5c13-41c1-b93d-9dfb97ad34ea',
    'color': 'blue',
    'value': '#Shallow Task'},
   {'id': '0dcc9608-348a-4983-a3aa-8fb621b

In [80]:
from toggl.TogglPy import Toggl
toggl = Toggl()
toggl.setAPIKey('ddfc096722a66679b0983e3b9b005c66')

In [85]:
client_dict = {
    'Becode':{
        'id': 50044442,
        'wid': 4392240,
        'name': 'Becode Pty Limited',
        'code': 'BE',
    },
    'EtymologyExplorer':{
        'id': 50044517,
        'wid': 4392240,
        'name': 'EtymologyExplorer',
        'code': 'EE',
    },
    'Frontline Conservation': {
        'id': 50044512,
        'wid': 4392240,
        'name': 'Frontline Conservation',
        'code': 'FCR',
    },
    'Mana Hawaiian Group': {
        'id': 50044439,
        'wid': 4392240,
        'name': 'Mana Hawaiian Group',
        'code': 'MANA',
    },
    'Might Could Studios': {
        'id': 50044510,
        'wid': 4392240,
        'name': 'Might Could Studios',
        'code': 'MC'
    },
    'Personal': {
        'id': 50044514,
        'wid': 4392240,
        'name': 'Personal',
        'code': 'PR',
    }
}

In [100]:
# def createProject(name):
data = { 
    "project": 
        { 
            "name": "Test", 
            "wid":4392240, 
#                 "template_id":10237, 
#                 "is_private":true, 
            "cid":50044510 
        }
    }

response = toggl.postRequest("https://www.toggl.com/api/v8/projects", parameters=data)
response

'{"data":{"id":165374732,"wid":4392240,"cid":50044510,"name":"Test","billable":false,"is_private":true,"active":true,"template":false,"at":"2020-11-25T00:40:08+00:00","color":"9","auto_estimates":false,"hex_color":"#990099"}}'

In [105]:
import json
json.loads('{"data":{"id":165374732,"wid":4392240,"cid":50044510,"name":"Test","billable":false,"is_private":true,"active":true,"template":false,"at":"2020-11-25T00:40:08+00:00","color":"9","auto_estimates":false,"hex_color":"#990099"}}')['data']['id']

165374732

In [ ]:
# Look for the project
# Create it if it doesn't exist
'{"project":{"name":"An awesome project","wid":777,"template_id":10237,"is_private":true,"cid":123397}}'
'https://api.track.toggl.com/api/v8/projects'

In [101]:
toggl.getClientProjects(50044510)

[{'id': 165374732,
  'wid': 4392240,
  'cid': 50044510,
  'name': 'Test',
  'billable': False,
  'is_private': True,
  'active': True,
  'template': False,
  'at': '2020-11-25T00:40:08+00:00',
  'created_at': '2020-11-25T00:40:08+00:00',
  'color': '9',
  'auto_estimates': False,
  'hex_color': '#990099'},
 {'id': 164648490,
  'wid': 4392240,
  'cid': 50044510,
  'name': 'Marketing Improvements',
  'billable': False,
  'is_private': True,
  'active': True,
  'template': False,
  'at': '2020-10-27T13:42:18+00:00',
  'created_at': '2020-10-27T13:42:18+00:00',
  'color': '3',
  'auto_estimates': False,
  'actual_hours': 6,
  'hex_color': '#e36a00'},
 {'id': 163396378,
  'wid': 4392240,
  'cid': 50044510,
  'name': 'Might Could Studios',
  'billable': False,
  'is_private': True,
  'active': True,
  'template': False,
  'at': '2020-09-15T21:03:07+00:00',
  'created_at': '2020-09-09T23:14:24+00:00',
  'color': '3',
  'auto_estimates': False,
  'actual_hours': 73,
  'hex_color': '#e36a00'}]

In [97]:
toggl.getClient(id=50044510)

{'id': 50044510,
 'wid': 4392240,
 'name': 'Might Could Studios',
 'at': '2020-09-15T21:01:31+00:00'}

In [92]:
'MC: Launch StSBC'.split(':')[1].strip()

'Launch StSBC'

In [ ]:
# myprojectpid = 10959693
# toggl.startTimeEntry("my description", myprojectpid)

### End

In [25]:
# artwork['Henriette_Ronner-Knip_The_cat_at_play_c._1860__1878.jpg']
props = art_rows[2].get_all_properties(); props
{**props, 'space': re.sub(',.+', '', props['space'])}

{'shape_form_defined_by': 'Value',
 'level_of_realism': 'Realism',
 'color_lightness': 'Darker',
 'line_width': 'No line',
 'texture': 'Smooth strokes',
 'color_saturation': 'Duller',
 'space': '3-D Pictorial Plane',
 'line_quality': 'No line',
 'question': 'Cats',
 'name': 'Henriëtte Ronner-Knip: The Cat at Play, 1860–1878'}